In [2]:
import torch
from datasets import load_dataset

dataset = load_dataset("mnist")

C:\Users\eshaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset mnist (C:\Users\eshaa\.cache\huggingface\datasets\mnist\mnist\1.0.0\fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4)
100%|██████████| 2/2 [00:00<00:00, 78.07it/s]


In [3]:
dataset['train'], dataset['test']

(Dataset({
     features: ['image', 'label'],
     num_rows: 60000
 }),
 Dataset({
     features: ['image', 'label'],
     num_rows: 10000
 }))

## Transform MNIST Dataset into variable sized images :D

In [79]:
from PIL import Image
from tqdm import tqdm
import numpy as np

x_train = []
y_train = []

for value in tqdm(dataset['train'], desc="Resizing x_train dataset"):
    moreUpscaleImage = value['image'].resize((64, 64), resample=Image.BOX)
    upscaleImage = value['image'].resize((48, 48), resample=Image.BOX)
    regularImage = value['image']

    moreUpscaleImage = torch.from_numpy(np.uint8(moreUpscaleImage)).flatten()
    upscaleImage = torch.from_numpy(np.uint8(upscaleImage)).flatten()
    regularImage = torch.from_numpy(np.uint8(regularImage)).flatten()

    x_train.extend([regularImage, upscaleImage, moreUpscaleImage]) 
    y_train.extend([value['label'], value['label'], value['label']])

x_test = []
y_test = []

for value in tqdm(dataset['test'], desc="Resizing x_test dataset"):
    moreUpscaleImage = value['image'].resize((64, 64), resample=Image.BOX)
    upscaleImage = value['image'].resize((48, 48), resample=Image.BOX)
    regularImage = value['image']

    moreUpscaleImage = torch.from_numpy(np.uint8(moreUpscaleImage)).flatten()
    upscaleImage = torch.from_numpy(np.uint8(upscaleImage)).flatten()
    regularImage = torch.from_numpy(np.uint8(regularImage)).flatten()

    x_test.extend([regularImage, upscaleImage, moreUpscaleImage]) 
    y_test.extend([value['label'], value['label'], value['label']])


Resizing x_train dataset:   0%|          | 0/60000 [00:00<?, ?it/s]C:\Users\eshaa\AppData\Local\Temp\ipykernel_9628\2604634332.py:9: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  moreUpscaleImage = value['image'].resize((64, 64), resample=Image.BOX)
C:\Users\eshaa\AppData\Local\Temp\ipykernel_9628\2604634332.py:10: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  upscaleImage = value['image'].resize((48, 48), resample=Image.BOX)
Resizing x_test dataset:   0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\eshaa\AppData\Local\Temp\ipykernel_9628\2604634332.py:24: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  moreUpscaleImage = value['image'].resize((64, 64), resample=Image.BOX)
C:\Users\eshaa\AppData\Local\Temp\ipykernel_9628\2604634332.py:25: DeprecationWarning: BOX is deprecated and will

In [80]:
from audioop import bias
from copy import deepcopy
import torch
from torch import nn
from torch import Tensor
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        return self.pe[x].squeeze(1)

class VNN (nn.Module):
    def __init__(self, dense_nn, weight_nn, bias_nn) -> None:
        super().__init__()
        self.d_model = (weight_nn[0].in_features-1)//2
        self.first_input = dense_nn[0].in_features

        self.dense_nn = dense_nn
        self.weight_nn = weight_nn 
        self.bias_nn = bias_nn 
        self.pos_enc = PositionalEncoding(self.d_model, dropout=0.1, max_len=5000)

    def generate_weight_vector (self, x, output_size):
        input_size = x.size(0)

        #* Weight Generation

        # Generate the weight vector
        argument_one = torch.arange(input_size).unsqueeze(1)
        
        argument_two = torch.arange(output_size)
        bias_argument = deepcopy(argument_two) 
        # Generate the repeat
        argument_one = argument_one.repeat(1, output_size).flatten()
        argument_two = argument_two.repeat(input_size)

        x_concat = x[argument_one].unsqueeze(1)
    
        # Positional Encoding + Concat
        argument_one = self.pos_enc(argument_one)
        argument_two = self.pos_enc(argument_two)
        bias_argument = self.pos_enc(bias_argument) 
        argument = torch.concat((argument_one, argument_two, x_concat), dim=1)

        # Send through the weight neural network
        weights = self.weight_nn(argument).view(input_size, output_size)

        out = torch.matmul(x, weights).unsqueeze(1)

        #* Bias Generation 
        argument = torch.concat((bias_argument, out), dim=1)
        bias = self.bias_nn(argument)
        out += bias

        return out.squeeze(1)

    def forward (self, x):
        x = self.generate_weight_vector(x, self.first_input)
        x = self.dense_nn(x)
        return x

## Neural Network Initialization

In [20]:
d_model = 32
weight_model = nn.Sequential(
    nn.Linear(65, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
) 

bias_model = nn.Sequential(
    nn.Linear(33, 10),
    nn.Tanh(),
    nn.Linear(10, 1)
)

dense_model = nn.Sequential(
    nn.Linear(128, 64),
    nn.Tanh(),
    nn.Linear(64,  10),
    nn.Softmax()
)

policy = VNN(dense_model, weight_model, bias_model)
policy(torch.randn(128))

C:\Users\eshaa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([0.0923, 0.0600, 0.1448, 0.1391, 0.1057, 0.0664, 0.1063, 0.1071, 0.0663,
        0.1120], grad_fn=<SoftmaxBackward0>)

## Training!

In [81]:
from tqdm import trange
from random import randint
import numpy as np

for _ in trange(len(x_train)):
    index = randint(0, len(x_train)-1)
    print(x_train[index])
    break
    
    pass
    

  0%|          | 0/180000 [00:00<?, ?it/s]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.uint8)
